# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

from markdown_it.rules_block import table
from onnxruntime.transformers.models.stable_diffusion.diffusion_models import BaseModel

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [1]:
import os
import chromadb
from dotenv import load_dotenv
from pydantic import BaseModel, Field, ConfigDict
from tavily import TavilyClient

from project.lib.agents import Agent
from project.lib.llm import LLM
from project.lib.messages import UserMessage, SystemMessage, BaseMessage
from project.lib.tooling import tool, Tool
from project.lib.evaluation import EvaluationReport
from project.lib.memory import LongTermMemory, VectorStoreManager, MemoryFragment

In [2]:
load_dotenv()

True

In [3]:
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")

In [4]:
# List all games
from tabulate import tabulate
response = collection.get()

game_list = []
for meta in response["metadatas"]:
    game_list.append([meta["Name"], meta["Genre"], meta["Publisher"], meta["Platform"], meta["YearOfRelease"]])

table = tabulate(
    game_list,
    headers=["Name", "Genre", "Publisher", "Platform", "YearOfRelease"],
    tablefmt="grid"
)

print(table)

+-----------------------------------------+----------------------+--------------------------------+--------------------------------------------+-----------------+
| Name                                    | Genre                | Publisher                      | Platform                                   |   YearOfRelease |
+=========================================+======================+================================+============================================+=================+
| Gran Turismo                            | Racing               | Sony Computer Entertainment    | PlayStation 1                              |            1997 |
+-----------------------------------------+----------------------+--------------------------------+--------------------------------------------+-----------------+
| Grand Theft Auto: San Andreas           | Action-adventure     | Rockstar Games                 | PlayStation 2                              |            2004 |
+---------------------

In [5]:
class Game(BaseModel):
    """Game information"""
    model_config = ConfigDict(
        frozen=True,
    )

    Platform: str = Field(
        ...,
        description="Platform name of the game",
        alias="Platform",
        examples=["Game Boy", "Playstation 5", "Xbox 360"]
    )
    Name: str = Field(
        ...,
        description="Name of the Game",
        alias="Name",
        examples=["Super Mario World", "Pokémon Red and Blue", "Gran Turismo"]
    )
    YearOfRelease: int = Field(
        ...,
        description="Year when that game was released for that platform",
        alias="YearOfRelease",
        examples=[1990, 2001, 2010]
    )
    Description: str = Field(
        ...,
        description="Additional details about the game",
        alias="Description",
        examples=["A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser."]
    )

In [6]:
@tool
def retrieve_game(query: str, num_of_results = 5) -> list[Game]:
    """
    Semantic search: Finds most results in the vector DB

    args:
    - query: a question about game industry.
    - num_of_results: number of results to return

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    retrieve_game_response = collection.query(query_texts=[query], n_results=int(num_of_results))

    if not (metadata := retrieve_game_response.get("metadatas", [[]])[0]):
        return metadata

    return [Game(**curr) for curr in metadata]

In [7]:
@tool
def evaluate_retrieval(question: str, retrieved_docs: list) -> EvaluationReport:
    """
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.

    args:
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database

    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """
    if not retrieved_docs:
        return EvaluationReport(
            useful=False,
            description="No documents were retrieved, so it's not possible to answer the question."
        )

    # Convert docs to serializable format (handles both Game objects and strings)
    docs_for_prompt = []
    for doc in retrieved_docs:
        if hasattr(doc, 'model_dump'):
            docs_for_prompt.append(doc.model_dump())
        else:
            docs_for_prompt.append(str(doc))

    llm_evaluator = LLM()
    messages: list[BaseMessage] = [
        SystemMessage(
            content=(
                "You are an expert in evaluating documents."
                "Your task is to evaluate if the information in the documents are enough to respond the user question."
                "Give a detailed explanation, so it's possible to take an action to accept it or not"
            )
        ),
        UserMessage(
            content=(
                f"""
                # Question: {question}
                # Documents: {docs_for_prompt}
                """
            )
        )
    ]

    evaluation_response = llm_evaluator.invoke(messages, response_format=EvaluationReport)

    if evaluation_response.content:
        return EvaluationReport.model_validate_json(evaluation_response.content)

    return EvaluationReport(
        useful=False,
        description="We were not able to evaluate the documents. Please try again."
    )

In [8]:
class SearchResult(BaseModel):
    """Search result"""
    model_config = ConfigDict(
        frozen=True,
    )

    title: str = Field(
        ...,
        description="Title of the search result",
        alias="title",
        examples=["Fifa 98"]
    )
    content: str = Field(
        ...,
        description="Content of the search result",
        alias="content",
        examples=["Fifa 98 was released in 1998."]
    )
    score: float = Field(
        default=0.0,
        description="Score of the search result",
        alias="score",
        examples=[0.9]
    )

In [9]:
@tool
def game_web_search(query: str) -> list[SearchResult]:
    """
    Semantic search: Finds most results in the vector DB

    args:
    - query: a question about game industry.
    """
    client = TavilyClient()
    web_search_response = client.search(query, num_results=5)

    return [SearchResult(**r) for r in web_search_response["results"]]

In [10]:
udaplay = Agent(
    model_name="gpt-4o-mini",
    instructions="""
    You are UdaPlay, an AI Research Agent for the video game industry.
    You are an Agentic RAG assistant that MUST use tools to answer user questions about video games.

    IMPORTANT: You MUST ALWAYS use tools to answer questions. NEVER answer from your own knowledge.

    ## Required Workflow (follow this for EVERY question):
    1. ALWAYS start by calling the `retrieve_game` tool to search the internal database
    2. After retrieval, ALWAYS call `evaluate_retrieval` to assess if the results answer the question
    3. If evaluation indicates results are enough, stop and answer the question, using the retrieved information.
    If not, proceed to the next step that involves searching the web using the `game_web_search` tool
    4. Only after using tools, provide your final answer based on the tool results

    Always explain your reasoning for tool selection and provide comprehensive answers based on tool results.
    """,
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
    temperature=0.1,
)


In [11]:
user_queries = [
    "When Minecraft was released?",
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for Playstation 5?",
]
session_id = "dcc120d2-3bcc-4543-afcb-207858823ab3"

for query in user_queries:
    run = udaplay.invoke(query, session_id=session_id)
    state = run.get_final_state()

    print("=" * 80)
    print(f"Query: {query}")
    print("=" * 80)

    # Show traces of tools being called in sequence
    print("\n📍 Tool Execution Trace:")
    tool_sequence = []
    for msg in state["messages"]:
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tc in msg.tool_calls:
                tool_sequence.append(tc.function.name)
                print(f"  → {tc.function.name}({tc.function.arguments[:100]}...)")

    if tool_sequence:
        print(f"\n🔗 Tool Sequence: {' → '.join(tool_sequence)}")

    print(f"\n✅ Final Response: {state['messages'][-1].content}")
    print("-" * 80)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep, State: {'user_query': 'When Minecraft was released?', 'instructions': '\n    You are UdaPlay, an AI Research Agent for the video game industry.\n    You are an Agentic RAG assistant that MUST use tools to answer user questions about video games.\n\n    IMPORTANT: You MUST ALWAYS use tools to answer questions. NEVER answer from your own knowledge.\n\n    ## Required Workflow (follow this for EVERY question):\n    1. ALWAYS start by calling the `retrieve_game` tool to search the internal database\n    2. After retrieval, ALWAYS call `evaluate_retrieval` to assess if the results answer the question\n    3. If evaluation indicates results are enough, stop and answer the question, using the retrieved information.\n    If not, proceed to the next step that involves searching the web using the `game_web_search` tool\n    4. Only after using tools, provide your final answer based on the tool results\n\n    Always e

### (Optional) Advanced

In [12]:
db = VectorStoreManager(os.getenv("OPENAI_API_KEY"))
_ = db.get_or_create_store("user_preferences")

long_term_memory = LongTermMemory(db)

In [13]:
from enum import StrEnum

class Namespace(StrEnum):
    CONVERSATION = "conversation"

def build_memory_registration_tool(ltm: LongTermMemory, id: str, namespace: Namespace):
    """
    Create a tool for agents to register new memories.

    This factory function creates a tool that allows AI agents to store new
    information about users in the long-term memory system. The tool is
    pre-configured with specific owner and namespace parameters.

    Args:
        ltm (LongTermMemory): The memory system instance to use
        id (str): User identifier for memory ownership
        namespace (Namespace): Namespace for organizing memories

    Returns:
        Tool: A configured tool for memory registration
    """
    def _register(content: str):
        ltm.register(
            MemoryFragment(
                content=content,
                owner=id,
                namespace=namespace
            )
        )
        return "Saved new memory"

    return Tool(
        func=_register,
        name="register_user_preference_memory",
        description=(
            "Register a new memory or preference about the user, "
            "so it can be useful later as context.\n"
            "Args:\n"
            "    content: The information to save"
        )
    )

def build_memory_search_tool(ltm: LongTermMemory, id: str, namespace: Namespace):
    """
    Create a tool for agents to search existing memories.

    This factory function creates a tool that allows AI agents to retrieve
    relevant memories from the long-term memory system based on semantic
    similarity to a search query.

    Args:
        ltm (LongTermMemory): The memory system instance to use
        id (str): User identifier for memory ownership
        namespace (str): Namespace to search within

    Returns:
        Tool: A configured tool for memory search
    """
    def _search(content:str):
        result = ltm.search(
            query_text=content,
            owner=id,
            namespace=namespace,
            limit=3,
        )
        return str(tuple(zip(result.fragments, result.metadata['distances'])))

    return Tool(
        func=_search,
        name="search_user_preference_memory",
        description=(
            "Search for a stored memory or preference about the user, "
            "so it's useful as a context.\n"
            "Args:\n"
            "    content: The information to look for"
        )
    )

In [14]:
user_id = "952034f3-f57b-4346-b5d4-672b53851712"

udaplay.tools.append(
    build_memory_registration_tool(long_term_memory, user_id, Namespace.CONVERSATION)
)
udaplay.tools.append(
    build_memory_search_tool(long_term_memory, user_id, Namespace.CONVERSATION)
)

udaplay.instructions += """
## Long-Term Memory
- Whenever the user shares a preference, personal information, or something important, use the `register_memory` tool to save it.
- Before answering questions about user preferences or history, use `search_memory` to retrieve relevant information.
- Use memory to personalize your responses.
"""

In [15]:
run = udaplay.invoke("my favorite game is God of War", session_id=session_id)
state = run.get_final_state()

print("-" * 80)
print(f"Response: {state['messages'][-1].content}")
print("-" * 80)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep, State: {'user_query': 'my favorite game is God of War', 'instructions': '\n    You are UdaPlay, an AI Research Agent for the video game industry.\n    You are an Agentic RAG assistant that MUST use tools to answer user questions about video games.\n\n    IMPORTANT: You MUST ALWAYS use tools to answer questions. NEVER answer from your own knowledge.\n\n    ## Required Workflow (follow this for EVERY question):\n    1. ALWAYS start by calling the `retrieve_game` tool to search the internal database\n    2. After retrieval, ALWAYS call `evaluate_retrieval` to assess if the results answer the question\n    3. If evaluation indicates results are enough, stop and answer the question, using the retrieved information.\n    If not, proceed to the next step that involves searching the web using the `game_web_search` tool\n    4. Only after using tools, provide your final answer based on the tool results\n\n    Always

In [16]:
run = udaplay.invoke("What is my favorite game?", session_id=session_id)
state = run.get_final_state()

print("-" * 80)
print(f"Response: {state['messages'][-1].content}")
print("-" * 80)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep, State: {'user_query': 'What is my favorite game?', 'instructions': '\n    You are UdaPlay, an AI Research Agent for the video game industry.\n    You are an Agentic RAG assistant that MUST use tools to answer user questions about video games.\n\n    IMPORTANT: You MUST ALWAYS use tools to answer questions. NEVER answer from your own knowledge.\n\n    ## Required Workflow (follow this for EVERY question):\n    1. ALWAYS start by calling the `retrieve_game` tool to search the internal database\n    2. After retrieval, ALWAYS call `evaluate_retrieval` to assess if the results answer the question\n    3. If evaluation indicates results are enough, stop and answer the question, using the retrieved information.\n    If not, proceed to the next step that involves searching the web using the `game_web_search` tool\n    4. Only after using tools, provide your final answer based on the tool results\n\n    Always expl

In [17]:
# Recreate using State Machine
import json

from typing import TypedDict, List, Optional, Union
from project.lib.state_machine import StateMachine, Step, EntryPoint, Termination, Run
from project.lib.messages import AIMessage, ToolMessage
from project.lib.tooling import ToolCall


In [18]:
class AgentState(TypedDict):
    user_query: str
    instructions: str
    messages: List[dict]
    current_tool_calls: Optional[List[ToolCall]]

In [19]:
def prepare_messages_step(state: AgentState) -> AgentState:
    """Step logic: Prepare messages for LLM consumption"""

    messages = [
        SystemMessage(content=state["instructions"]),
        UserMessage(content=state["user_query"])
    ]

    return {
        "messages": messages
    }

In [20]:
tools_list = [retrieve_game, evaluate_retrieval, game_web_search]

In [21]:
def llm_step(state: AgentState) -> AgentState:
    """Step logic: Process the current state through the LLM"""

    # Initialize LLM
    llm = LLM(
        model="gpt-4o-mini",
        temperature=0.1,
        tools=tools_list,
    )

    llm_response = llm.invoke(state["messages"])
    tool_calls = llm_response.tool_calls if llm_response.tool_calls else None

    # Create AI message with content and tool calls
    ai_message = AIMessage(content=llm_response.content, tool_calls=tool_calls)

    return {
        "messages": state["messages"] + [ai_message],
        "current_tool_calls": tool_calls
    }

In [22]:
def tool_step(state: AgentState) -> AgentState:
    """Step logic: Execute any pending tool calls"""
    tool_calls = state["current_tool_calls"] or []
    tool_messages = []

    for call in tool_calls:
        # Access tool call data correctly
        function_name = call.function.name
        function_args = json.loads(call.function.arguments)
        tool_call_id = call.id
        # Find the matching tool
        tool = next((t for t in tools_list if t.name == function_name), None)
        if tool:
            result = tool(**function_args)
            # Convert result to JSON-serializable format
            if hasattr(result, 'model_dump'):
                content = json.dumps(result.model_dump())
            elif isinstance(result, list) and result and hasattr(result[0], 'model_dump'):
                content = json.dumps([r.model_dump() for r in result])
            else:
                content = json.dumps(result)
            tool_messages.append(
                ToolMessage(
                    content=content,
                    tool_call_id=tool_call_id,
                    name=function_name,
                )
            )

    # Clear tool calls and add results to messages
    return {
        "messages": state["messages"] + tool_messages,
        "current_tool_calls": None
    }

In [23]:
workflow = StateMachine[AgentState](AgentState)

In [24]:
# Create steps
entry = EntryPoint[AgentState]()
message_prep = Step[AgentState]("message_prep", prepare_messages_step)
llm_processor = Step[AgentState]("llm_processor", llm_step)
tool_executor = Step[AgentState]("tool_executor", tool_step)
termination = Termination[AgentState]()

workflow.add_steps(
    [
        entry,
        message_prep,
        llm_processor,
        tool_executor,
        termination
    ]
)

In [25]:
# Add transitions
workflow.connect(entry, message_prep)
workflow.connect(message_prep, llm_processor)

# Transition based on whether there are tool calls
def check_tool_calls(state: AgentState) -> Union[Step[AgentState], str]:
    """Transition logic: Check if there are tool calls"""
    if state.get("current_tool_calls"):
        return tool_executor
    return termination

# Routing: If tool calls -> tool_executor
workflow.connect(
    source=llm_processor,
    targets=[tool_executor, termination],
    condition=check_tool_calls
)

# Looping: Go back to llm after tool execution
workflow.connect(
    source=tool_executor,
    targets=llm_processor
)

In [26]:
initial_state: AgentState = {
    "user_query": "When was released the first Madden NFL game?",
    "instructions": ("""
    You are UdaPlay, an AI Research Agent for the video game industry.
    You are an Agentic RAG assistant that MUST use tools to answer user questions about video games.

    IMPORTANT: You MUST ALWAYS use tools to answer questions. NEVER answer from your own knowledge.

    ## Required Workflow (follow this for EVERY question):
    1. ALWAYS start by calling the `retrieve_game` tool to search the internal database
    2. After retrieval, ALWAYS call `evaluate_retrieval` to assess if the results answer the question
    3. If evaluation indicates results are enough, stop and answer the question, using the retrieved information.
    If not, proceed to the next step that involves searching the web using the `game_web_search` tool
    4. Only after using tools, provide your final answer based on the tool results

    Always explain your reasoning for tool selection and provide comprehensive answers based on tool results.
    """),
    "messages": [],
}

In [28]:
run_object = workflow.run(initial_state)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep, State: {'user_query': 'When was released the first Madden NFL game?', 'instructions': '\n    You are UdaPlay, an AI Research Agent for the video game industry.\n    You are an Agentic RAG assistant that MUST use tools to answer user questions about video games.\n\n    IMPORTANT: You MUST ALWAYS use tools to answer questions. NEVER answer from your own knowledge.\n\n    ## Required Workflow (follow this for EVERY question):\n    1. ALWAYS start by calling the `retrieve_game` tool to search the internal database\n    2. After retrieval, ALWAYS call `evaluate_retrieval` to assess if the results answer the question\n    3. If evaluation indicates results are enough, stop and answer the question, using the retrieved information.\n    If not, proceed to the next step that involves searching the web using the `game_web_search` tool\n    4. Only after using tools, provide your final answer based on the tool results

In [34]:
state = run_object.get_final_state()

print("=" * 80)
print("Tool Execution Trace:")
tool_sequence = []
for msg in state["messages"]:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tc in msg.tool_calls:
            tool_sequence.append(tc.function.name)
            print(f"  → {tc.function.name}({tc.function.arguments[:100]}...)")

if tool_sequence:
    print(f"\nTool Sequence: {' → '.join(tool_sequence)}")

print(f"\nFinal Response: {state['messages'][-1].content}")
print("-" * 80)

Tool Execution Trace:
  → retrieve_game({"query":"first Madden NFL game","num_of_results":"5"}...)
  → evaluate_retrieval({"question":"When was released the first Madden NFL game?","retrieved_docs":"[{\"Platform\": \"GameC...)
  → game_web_search({"query":"first Madden NFL game release date"}...)

Tool Sequence: retrieve_game → evaluate_retrieval → game_web_search

Final Response: The first Madden NFL game, originally titled "John Madden Football," was released on June 1, 1988, for the Apple II. This game marked the beginning of a long-running series that has become a staple in sports video gaming.
--------------------------------------------------------------------------------
